![cabecera](./data/Proyecto.jpg)

# 💄 Análisis Exploratorio de Datos: Nala - De los leus al euro

## 1. Business Case & Data Collection

### 1.1 Business Understanding
La marca **Nala**, conocida por su propuesta de cosmética ética y sostenible, se vende tanto en **España** como en **Rumanía**. El objetivo de este análisis es explorar cómo se adapta su precio a las realidades económicas locales y determinar si Nala es igual de accesible en ambos mercados o se percibe como una marca de lujo en uno de ellos.

### 1.2 Hipótesis
- **Hipótesis principal**: Los precios de Nala en Rumanía, considerando el poder adquisitivo local, son significativamente menos accesibles que en España.
- **Hipótesis secundaria**: Existen diferencias en la estructura de precios y mix de productos entre ambos mercados.

### 1.3 Plan de Acción
- Analizar distribución y estadísticas de precios por mercado
- Calcular el "índice de accesibilidad Nala" relacionando precios con salarios mínimos
- Identificar categorías con mayores diferencias de precio

### 1.4 Fuentes de Datos
- Datos de productos Nala en España (`nala_es.csv`)
- Datos de productos Nala en Rumanía (`nala_ro.csv`)

## 2. Data Understanding

### 2.1 Exploración Inicial

In [2]:
# Código para carga y exploración inicial
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Configuración
plt.rcParams['figure.dpi'] = 120
sns.set(style='whitegrid')

# Carga de datos
df_es = pd.read_csv('./data/nala_es.csv')
df_ro = pd.read_csv('./data/nala_ro.csv')

### 2.2 Tabla de Variables

| Variable          | Tipo       | Descripción                    | Uso en Análisis            |
|-------------------|------------|--------------------------------|-----------------------------|
| SKU               | Categórica | Identificador único del producto | Identificación            |
| gramos/ml         | Numérica   | Cantidad del producto           | Normalización de precios  |
| nombre            | Categórica | Nombre del producto             | Descripción               |
| categoria_general | Categórica | Categoría principal             | Agrupación                |
| categoria         | Categórica | Categoría específica            | Análisis detallado        |
| precio            | Numérica   | Precio del producto             | Variable principal        |
| ingrediente_clave | Categórica | Ingrediente principal           | Segmentación              |
| país              | Categórica | Mercado (ES/RO)                 | Comparación               |
| url               | Categórica | Enlace del producto             | Referencia                |


# 3. Data Cleaning
### 3.1 Limpieza y Preprocesamiento

In [4]:
### Eliminamos y limpiamos los 2 dataset y nos quedamos solo con lo necesario, trabajaremos solo con productos individuales


# Función para limpieza consistente
def limpiar_datos(df, pais, keywords_pack):
    print(f"\nLimpieza para {pais}:")
    
    # Copia del dataframe
    df_clean = df.copy()
    
    # Verificar valores nulos
    print(f"Valores nulos antes de limpieza: {df_clean.isnull().sum().sum()}")
    
    # Limpiar gramos/ml - manejar valores nulos y convertir a entero
    df_clean["gramos/ml"] = df_clean["gramos/ml"].fillna(0).astype(int)
    print(f"Gramos/ML limpiados - Rango: {df_clean['gramos/ml'].min()} - {df_clean['gramos/ml'].max()}")
    
    # Filtrar categoría "Otro" si existe
    if "categoria" in df_clean.columns:
        df_clean = df_clean[df_clean["categoria"] != "Otro"]
        print(f"Productos después de quitar 'Otro': {len(df_clean)}")
    
    # Filtrar packs y sets
    initial_count = len(df_clean)
    for keyword in keywords_pack:
        df_clean = df_clean[~df_clean["nombre"].str.contains(keyword, case=False, na=False)]
    print(f"Packs filtrados: {initial_count - len(df_clean)} productos eliminados")
    
    # Agregar columna de país
    df_clean['país'] = pais
    
    return df_clean

# Aplicar limpieza
keywords_es = ["pack", "set", "rutina"]
keywords_ro = ["pachet", "pack", "set", "rutina"]

df_es_clean = limpiar_datos(df_es, "España", keywords_es)
df_ro_clean = limpiar_datos(df_ro, "Rumania", keywords_ro)

# Combinar datasets
df_es_ro = pd.concat([df_es_clean, df_ro_clean], ignore_index=True)

# Limpieza final del dataset combinado
keywords_combinadas = ["pack", "set", "rutina", "pachet", "kit", "combo"]
inicial = len(df_es_ro)

for keyword in keywords_combinadas:
    df_es_ro = df_es_ro[~df_es_ro["nombre"].str.contains(keyword, case=False, na=False)]

# Guardar dataset limpio
df_es_ro.to_csv("./data/nala_es_ro.csv", index=False)


Limpieza para España:
Valores nulos antes de limpieza: 64
Gramos/ML limpiados - Rango: 0 - 600
Productos después de quitar 'Otro': 422
Packs filtrados: 41 productos eliminados

Limpieza para Rumania:
Valores nulos antes de limpieza: 80
Gramos/ML limpiados - Rango: 0 - 600
Productos después de quitar 'Otro': 418
Packs filtrados: 76 productos eliminados


# 4. Análisis Exploratorio

📋 Preguntas de Investigación

### Análisis General de Mercado
- ¿Cómo se distribuyen los precios de los productos Nala en España vs Rumanía?

- ¿Existe una diferencia significativa en el rango de precios entre ambos mercados?

- ¿Cuál es la dispersión de precios en cada país?

### Análisis por Categorías
- ¿Qué categorías de productos tienen los precios más altos y más bajos?

- ¿Cómo varían los precios entre categorías generales?

- ¿Hay categorías donde la diferencia de precio entre países es más pronunciada?

### Análisis de Productos
- ¿Cuáles son los productos más caros y más económicos en cada mercado?

- ¿Cómo se relaciona el precio con la cantidad del producto?

- ¿Existen productos idénticos con precios diferentes entre países?

### Análisis de Accesibilidad
- Considerando los salarios mínimos, ¿es Nala igual de accesible en ambos mercados?

### Estadistica Descriptiva

In [12]:
print("📈 ESTADÍSTICAS DESCRIPTIVAS POR PAÍS")
print("=" * 50)

# Estadísticas básicas
stats_pais = df_es_ro.groupby('país')['precio'].describe()
print(stats_pais)


print("\n🔍 ANÁLISIS COMPARATIVO (precios convertidos):")

# Tasa de conversión
tasa = 5.03  # 1 euro = 5.03 lei

# Rango España (igual)
print(f"Rango de precios España: €{df_es_clean['precio'].min():.2f} - €{df_es_clean['precio'].max():.2f}")

# Rango Rumanía convertido a euros
rango_min_ro = df_ro_clean['precio'].min() / tasa
rango_max_ro = df_ro_clean['precio'].max() / tasa
print(f"Rango de precios Rumanía: €{rango_min_ro:.2f} - €{rango_max_ro:.2f} (convertido desde LEI)")


# Conteo por categoría
print("\n📊 PRODUCTOS POR CATEGORÍA:")
categoria_counts = df_es_ro.groupby(['país', 'categoria_general']).size().unstack(fill_value=0)
print(categoria_counts)

📈 ESTADÍSTICAS DESCRIPTIVAS POR PAÍS
         count       mean        std   min   25%   50%   75%    max
país                                                               
España   381.0   7.831627   3.678423  2.35   4.9   7.9   9.9   22.9
Rumania  342.0  32.173538  18.339349  6.45  19.9  29.9  39.9  119.9

🔍 ANÁLISIS COMPARATIVO (precios convertidos):
Rango de precios España: €2.35 - €22.90
Rango de precios Rumanía: €1.28 - €23.84 (convertido desde LEI)

📊 PRODUCTOS POR CATEGORÍA:
categoria_general  Cabello  Corporal  Ducha y Baño  Rostro
país                                                      
España                  30       153           122      76
Rumania                 30       132           108      72


In [6]:
# Gráfica general de análisis de mercados - VISIÓN GLOBAL

# Parámetros de conversión
tipo_cambio = 5.03  # 1 EUR = 5.03 LEI

# Crear la columna 'gama' con conversión
def clasificar_gama(precio, pais):
    if pais == "España":
        return "Baja" if precio <= 8 else "Media" if precio <= 15 else "Alta"
    else:  # Rumanía
        # Convertir límites de España a LEI usando el tipo de cambio
        limite_baja = 8 * tipo_cambio
        limite_media = 15 * tipo_cambio
        return "Baja" if precio <= limite_baja else "Media" if precio <= limite_media else "Alta"

df_es_ro["gama"] = df_es_ro.apply(lambda row: clasificar_gama(row["precio"], row["país"]), axis=1)

# Crear columna de precio en EUR para toda la visualización
df_es_ro['precio_eur'] = df_es_ro.apply(
    lambda row: row['precio'] if row['país'] == 'España' else row['precio'] / tipo_cambio, 
    axis=1
)

# Gráfica sunburst con precios en EUR
fig = px.sunburst(
    df_es_ro,
    path=['país', 'categoria_general', 'gama'],
    values='precio_eur',  # Usar precios en EUR
    color='precio_eur',   # Color basado en EUR
    color_continuous_scale='RdYlBu_r',
    title='<b>VISIÓN GLOBAL NALA</b><br>Distribución del Portfolio por País, Categoría y Gama de Precio<br><sub>Precios convertidos a EUR (1€ = 5.03 LEI)</sub>',
    width=800,
    height=800
)

fig.update_layout(
    template='plotly_white',
    font=dict(size=12)
)

fig.show()

## Distribución de Precios

In [7]:
# Histograma comparativo
fig_hist = px.histogram(
    df_es_ro, 
    x='precio', 
    color='país', 
    nbins=30,
    title='Distribución de Precios - España vs Rumanía',
    barmode='overlay',
    opacity=0.7
)
fig_hist.show()

HISTOGRAMA:

• España: Distribución concentrada €0-€15 (productos accesibles)

• Rumanía: Distribución más dispersa con cola larga hacia precios altos

• Insight: En Rumanía hay más variabilidad de precios.

In [8]:
# Gráfica interactiva de distribución de precios con categorías
fig_distribucion = px.histogram(
    df_es_ro,
    x='precio',
    color='país',
    facet_col='categoria_general',
    nbins=20,
    title='📊 Distribución de Precios por Categoría General - España vs Rumanía',
    barmode='overlay',
    opacity=0.7
)

fig_distribucion.update_layout(
    height=600,
    showlegend=True,
    xaxis_title="Precio",
    yaxis_title="Frecuencia"
)

fig_distribucion.show()

In [9]:
# Boxplot comparativo
fig_box = px.box(
    df_es_ro, 
    x='país', 
    y='precio',
    title='Comparación de Precios por País',
    color='país'
)
fig_box.show()

BOXPLOT:

España: Rango intercuartil estrecho (€4.9 - €9.9), mediana €7.9.

Rumanía: Mayor dispersión (19.9 - 53.4 LEI), más outliers.

Conclusión: Los precios en Rumanía son más variables y extremos.

In [10]:
# Violin plot por categoría
fig_violin = px.violin(
    df_es_ro,
    x='categoria_general',
    y='precio',
    color='país',
    box=True,
    points=False,
    title='Distribución de Precios por Categoría General'
)
fig_violin.show()

In [ ]:
# Aplica conversión de moneda a todo el dataset
tipo_cambio = 5.03  # 1 EUR = 5.03 RON (LEI)

df_es_ro_eur = df_es_ro.assign(
    precio_eur=lambda x: x.apply(
        lambda row: row['precio'] if row['país'] == 'España' else row['precio'] / tipo_cambio, 
        axis=1
    )
)

# Calcular precios promedio en EUR por categoría específica y país
precio_promedio_categorias_eur = df_es_ro_eur.groupby(['categoria', 'país'])['precio_eur'].agg(['mean', 'count']).reset_index()
precio_promedio_categorias_eur.columns = ['categoria', 'país', 'precio_promedio_eur', 'cantidad_productos']

# Obtener las 35 categorías con más productos
top_35_categorias = precio_promedio_categorias_eur.groupby('categoria')['cantidad_productos'].sum().nlargest(35).index
precio_promedio_top35 = precio_promedio_categorias_eur[
    precio_promedio_categorias_eur['categoria'].isin(top_35_categorias)
]

# Gráfica de barras interactiva en EUR con las 35 categorías principales
fig_barras_eur = px.bar(
    precio_promedio_top35,
    x='categoria',
    y='precio_promedio_eur',
    color='país',
    barmode='group',
    title='Precio Promedio por Categoría Específica - España vs Rumanía (EUR) - Top 35 Categorías',
    hover_data=['cantidad_productos'],
    labels={
        'categoria': 'Categoría Específica',
        'precio_promedio_eur': 'Precio Promedio (EUR)',
        'país': 'País',
        'cantidad_productos': 'Nº Productos'
    },
    color_discrete_map={
        'España': '#1f77b4',
        'Rumania': '#ff7f0e'
    }
)

fig_barras_eur.update_layout(
    height=700,
    xaxis_tickangle=-45,
    showlegend=True,
    xaxis_title="Categoría Específica",
    yaxis_title="Precio Promedio (EUR)",
    template='plotly_white'
)

# Mejorar tooltip
fig_barras_eur.update_traces(
    hovertemplate=(
        "<b>%{x}</b><br>"
        "País: %{marker.color}<br>"
        "Precio promedio: %{y:.2f}€<br>"
        "Productos: %{customdata[0]}"
        "<extra></extra>"
    )
)

fig_barras_eur.show()

Total de categorías mostradas: 35
Total de productos en estas categorías: 711


# Análisis de Accesibilidad


In [11]:
# 🎯 ANÁLISIS DE ACCESIBILIDAD 

print("\n🎯 ANÁLISIS DE ACCESIBILIDAD")
print("=" * 40)

# Parámetros económicos 2025
tipo_cambio = 5.03  # EUR a RON
salario_min_es = 1184   # EUR/mes (SMI España 2025)
salario_min_ro = 4050   # RON/mes (Salario mínimo Rumanía 2025)

print(f"💱 Tipo de cambio: 1 EUR = {tipo_cambio} RON")
print(f"💰 Salario mínimo España: €{salario_min_es}/mes")
print(f"💰 Salario mínimo Rumanía: {salario_min_ro} RON/mes (€{salario_min_ro/tipo_cambio:.2f})")

# Convertir precios Rumanía a EUR para comparación directa
df_ro_clean['precio_eur'] = df_ro_clean['precio'] / tipo_cambio

# Calcular métricas de accesibilidad
def calcular_accesibilidad(df, salario_min, columna_precio='precio'):
    """Calcula métricas de accesibilidad"""
    precio_mediano = df[columna_precio].median()
    productos_salario = salario_min / precio_mediano
    porcentaje_salario = (precio_mediano / salario_min) * 100
    
    return {
        'precio_mediano': precio_mediano,
        'productos_salario': productos_salario,
        'porcentaje_salario': porcentaje_salario
    }

# Calcular para España
acc_es = calcular_accesibilidad(df_es_clean, salario_min_es)

# Calcular para Rumanía (en EUR)
salario_min_ro_eur = salario_min_ro / tipo_cambio
acc_ro = calcular_accesibilidad(df_ro_clean, salario_min_ro_eur, 'precio_eur')

print(f"\n📊 MÉTRICAS DE ACCESIBILIDAD:")
print(f"ESPAÑA:")
print(f"  • Precio mediano: €{acc_es['precio_mediano']:.2f}")
print(f"  • Productos que se pueden comprar con salario mínimo: {acc_es['productos_salario']:.0f}")
print(f"  • % del salario mínimo por producto: {acc_es['porcentaje_salario']:.1f}%")

print(f"RUMANÍA:")
print(f"  • Precio mediano: €{acc_ro['precio_mediano']:.2f}")
print(f"  • Productos que se pueden comprar con salario mínimo: {acc_ro['productos_salario']:.0f}")
print(f"  • % del salario mínimo por producto: {acc_ro['porcentaje_salario']:.1f}%")

# CREAR EL ÍNDICE DE ACCESIBILIDAD QUE FALTABA
indice_accesibilidad = {
    'España': acc_es['porcentaje_salario'],
    'Rumanía': acc_ro['porcentaje_salario']
}

print(f"\n📈 ÍNDICE DE ACCESIBILIDAD (menor % = más accesible):")
for pais, indice in indice_accesibilidad.items():
    print(f"  • {pais}: {indice:.1f}%")

# Ratio de accesibilidad
ratio_accesibilidad = acc_es['productos_salario'] / acc_ro['productos_salario']
print(f"\n⚖️ RATIO DE ACCESIBILIDAD (ES/RO): {ratio_accesibilidad:.2f}x")

# Interpretación
if ratio_accesibilidad > 1:
    print("📍 INTERPRETACIÓN: Los productos son MÁS accesibles en España")
else:
    print("📍 INTERPRETACIÓN: Los productos son MÁS accesibles en Rumanía")


🎯 ANÁLISIS DE ACCESIBILIDAD
💱 Tipo de cambio: 1 EUR = 5.03 RON
💰 Salario mínimo España: €1184/mes
💰 Salario mínimo Rumanía: 4050 RON/mes (€805.17)

📊 MÉTRICAS DE ACCESIBILIDAD:
ESPAÑA:
  • Precio mediano: €7.90
  • Productos que se pueden comprar con salario mínimo: 150
  • % del salario mínimo por producto: 0.7%
RUMANÍA:
  • Precio mediano: €5.94
  • Productos que se pueden comprar con salario mínimo: 135
  • % del salario mínimo por producto: 0.7%

📈 ÍNDICE DE ACCESIBILIDAD (menor % = más accesible):
  • España: 0.7%
  • Rumanía: 0.7%

⚖️ RATIO DE ACCESIBILIDAD (ES/RO): 1.11x
📍 INTERPRETACIÓN: Los productos son MÁS accesibles en España


In [13]:
# Preparar datos para visualización en EUR
df_es_ro_eur = df_es_ro.copy()
df_es_ro_eur['precio_eur'] = df_es_ro_eur.apply(
    lambda x: x['precio'] if x['país'] == 'España' else x['precio'] / tipo_cambio, 
    axis=1
)

# Boxplot en euros
fig_box_eur = px.box(
    df_es_ro_eur, 
    x='país', 
    y='precio_eur',
    title='Comparación de Precios en Euros - España vs Rumanía',
    color='país',
    labels={'precio_eur': 'Precio (EUR)', 'país': 'País'}
)
fig_box_eur.show()

In [14]:
# Histograma en euros (CÓDIGO COMPLETO Y CORREGIDO)
fig_hist_eur = px.histogram(
    df_es_ro_eur,
    x='precio_eur',
    color='país',
    nbins=30,
    title='Distribución de Precios en Euros - España vs Rumanía',
    barmode='overlay',
    opacity=0.7,
    labels={'precio_eur': 'Precio (EUR)'}
)
fig_hist_eur.show()

In [18]:
# Definir tipo de cambio para evitar errores
tipo_cambio = 5.03  # 1 euro = 5.03 lei

# --- Recalcular productos comprables (mismas métricas del análisis anterior) ---

# Precio mediano España
precio_mediano_es = df_es_clean['precio'].median()

# Precio mediano Rumanía (convertido a EUR)
df_ro_clean['precio_eur'] = df_ro_clean['precio'] / tipo_cambio
precio_mediano_ro = df_ro_clean['precio_eur'].median()

# Salarios
salario_min_es = 1184
salario_min_ro = 4050 / tipo_cambio  # convertir a euros

# Productos comprables
productos_salario_es = salario_min_es / precio_mediano_es
productos_salario_ro = salario_min_ro / precio_mediano_ro

# --- Crear DataFrame para el gráfico ---
accesibilidad_data = pd.DataFrame({
    'País': ['España', 'Rumanía'],
    'Productos_comprables': [productos_salario_es, productos_salario_ro],
    'Salario_mensual': [salario_min_es, salario_min_ro]
})

# --- Gráfico ---
fig_accesibilidad = px.bar(
    accesibilidad_data,
    x='País',
    y='Productos_comprables',
    title='Accesibilidad Nala - Productos comprables con un salario mínimo',
    labels={'Productos_comprables': 'Número de productos', 'País': ''},
    color='País',
    text_auto=True,
)

fig_accesibilidad.update_layout(
    showlegend=False,
    template='plotly_white',
    yaxis_title='Número de productos'
)

fig_accesibilidad.show()



Con tu salario mensual, puedes comprar 150 productos en españa y 135 productos en Rumania productos Nala

In [19]:
df_indice = pd.DataFrame({
    'País': list(indice_accesibilidad.keys()),
    'Índice Accesibilidad': list(indice_accesibilidad.values())
})

fig_indice = px.bar(
    df_indice,
    x='País',
    y='Índice Accesibilidad',
    title='Índice de Accesibilidad Nala (% del salario mínimo diario)',
    labels={'Índice Accesibilidad': '% del salario diario'},
    color='País',
)

# Agregar los números en cada barra
fig_indice.update_traces(
    texttemplate='%{y:.2f}%',  # Mostrar el valor con 2 decimales y símbolo %
    textposition='outside'      # Posicionar el texto fuera de las barras
)

fig_indice.update_layout(showlegend=False)
fig_indice.show()

## Análisis Precio vs Cantidad

In [21]:
# Cargar el DataFrame combinado desde el archivo para evitar errores
nala_es_ro = pd.read_csv('./data/nala_es_ro.csv')

# Luego calcular precio por ml
nala_es_ro['precio_por_ml'] = nala_es_ro['precio'] / nala_es_ro['gramos/ml']

# Filtrar solo productos con cantidad válida (> 0)
nala_es_ro_cantidad_valida = nala_es_ro[nala_es_ro['gramos/ml'] > 0]

In [43]:
### Relacion precio VS cantidad
tipo_cambio = 5.03  # 1 EUR = 5.03 LEI

# Crear copia "fixed" para mantener tu nombre original manera para evitar warning 
nala_es_ro_cantidad_valida_fixed = nala_es_ro_cantidad_valida.copy()

# Asegurar que existen las columnas convertidas a euros
if 'precio_eur' not in nala_es_ro_cantidad_valida_fixed.columns:
    nala_es_ro_cantidad_valida_fixed['precio_eur'] = nala_es_ro_cantidad_valida_fixed.apply(
        lambda row: row['precio'] if row['país'] == 'España' else row['precio'] / tipo_cambio,
        axis=1
    )

if 'precio_por_ml_eur' not in nala_es_ro_cantidad_valida_fixed.columns:
    nala_es_ro_cantidad_valida_fixed['precio_por_ml_eur'] = (
        nala_es_ro_cantidad_valida_fixed['precio_eur'] / nala_es_ro_cantidad_valida_fixed['gramos/ml']
    )

# --- 2️⃣ Crear el scatter plot ---
fig_scatter = px.scatter(
    nala_es_ro_cantidad_valida_fixed,
    x='gramos/ml',
    y='precio_eur',
    color='país',
    hover_data=['nombre', 'categoria_general', 'precio_por_ml_eur'],
    title='Relación Precio vs Cantidad - España vs Rumanía',
    labels={
        'gramos/ml': 'Cantidad (gramos/ml)', 
        'precio_eur': 'Precio (EUR)',
        'categoria_general': 'Categoría',
        'país': 'País',
        'precio_por_ml_eur': 'Precio por ml/g (EUR)'
    },
    opacity=0.7
)

fig_scatter.update_layout(
    template='plotly_white',
    showlegend=True,
    xaxis_title="Cantidad (gramos/ml)",
    yaxis_title="Precio (EUR)",
    height=500
)

# --- 3️⃣ Tooltip personalizado ---
fig_scatter.update_traces(
    hovertemplate=(
        "<b>%{customdata[0]}</b><br>"
        "Categoría: %{customdata[1]}<br>"
        "Cantidad: %{x}g/ml<br>"
        "Precio total: %{y:.2f}€<br>"
        "Precio por unidad: %{customdata[2]:.4f}€<br>"
        "País: %{marker.color}"
        "<extra></extra>"
    )
)

# --- 4️⃣ Mostrar gráfico ---
fig_scatter.show()


In [28]:
# Usamos .assign() para evitar el warning 
# Este método retorna automáticamente
# CONVERSIÓN CORRECTA LEI A EURO


tipo_cambio = 5.03  # 1 EUR = 5.03 RON (LEI)

nala_es_ro_cantidad_valida_fixed = nala_es_ro_cantidad_valida.assign(
    precio_eur=lambda x: x.apply(
        lambda row: row['precio'] if row['país'] == 'España' else row['precio'] / tipo_cambio, 
        axis=1
    ),
    precio_por_ml_eur=lambda x: x.apply(
        lambda row: row['precio_por_ml'] if row['país'] == 'España' else row['precio_por_ml'] / tipo_cambio, 
        axis=1
    )
)

# Gráfica
fig_bubbles_simple = px.scatter(
    nala_es_ro_cantidad_valida_fixed,
    x='gramos/ml',
    y='precio_eur',
    size='precio_por_ml_eur',
    color='categoria_general',
    facet_col='país',
    hover_data={
        'precio_por_ml_eur': ':.3f', 
        'precio_eur': ':.2f',
        'precio': ':.2f'
    },
    title='Precio vs Cantidad por País y Categoría<br><sub>Tamaño = Precio por ML | Precios en EUR (1 EUR = 5.03 RON)</sub>',
    size_max=20,
    opacity=0.6,
    labels={
        'gramos/ml': 'Cantidad (gramos/ml)',
        'precio_eur': 'Precio (EUR)',
        'precio_por_ml_eur': 'Precio por ML (EUR/ml)',
        'categoria_general': 'Categoría General'
    }
)

fig_bubbles_simple.update_layout(template='plotly_white', showlegend=True)
fig_bubbles_simple.show()

In [45]:
# Precio total vs Precio por unidad
df_precio_unidad = pd.DataFrame()

# Precio por unidad
precio_unidad = nala_es_ro_cantidad_valida_fixed[['país', 'precio_por_ml_eur']].copy()
precio_unidad['tipo'] = 'Precio por unidad'
precio_unidad['valor'] = precio_unidad['precio_por_ml_eur']

# Precio total
precio_total = nala_es_ro_cantidad_valida_fixed[['país', 'precio_eur']].copy()
precio_total['tipo'] = 'Precio total' 
precio_total['valor'] = precio_total['precio_eur']

# Combinar
df_precio_unidad = pd.concat([precio_unidad, precio_total])

fig_precio_unidad = px.box(
    df_precio_unidad,
    x='país',
    y='valor',
    color='tipo',
    title='Comparación de Precio Total vs Precio por Unidad<br><sub>Todos los precios en EUR</sub>',
    labels={'valor': 'Precio (EUR)', 'país': 'País'}
)

fig_precio_unidad.update_layout(template='plotly_white')
fig_precio_unidad.show()

## Productos Extremos y Top Categorías

In [30]:
# Top 5 productos más caros por país 
top_5_es = nala_es_ro[nala_es_ro["país"] == "España"].nlargest(8, "precio").copy()
top_5_ro = nala_es_ro[nala_es_ro["país"] == "Rumania"].nlargest(8, "precio").copy()

# Convertir precios de Rumanía a EUR para comparación
tipo_cambio = 5.03
top_5_ro["precio_eur"] = top_5_ro["precio"] / tipo_cambio

#  Crear etiquetas con SKU para usar en el eje Y
top_5_es["label"] = top_5_es["nombre"] + " (SKU " + top_5_es["SKU"].astype(str) + ")"
top_5_ro["label"] = top_5_ro["nombre"] + " (SKU " + top_5_ro["SKU"].astype(str) + ")"

fig = go.Figure()

# España - barras azules
fig.add_trace(go.Bar(
    y=top_5_es["label"],
    x=top_5_es["precio"],
    orientation="h",
    name="España",
    marker_color="#1f77b4",
    hovertemplate="<b>%{y}</b><br>Precio: €%{x:.2f}<br>Categoría: " +
                top_5_es["categoria"] + "<extra></extra>"
))

# Rumanía - barras naranjas
fig.add_trace(go.Bar(
    y=top_5_ro["label"],
    x=top_5_ro["precio_eur"],
    orientation="h",
    name="Rumanía (EUR)",
    marker_color="#ff7f0e",
    hovertemplate="<b>%{y}</b><br>Precio: €%{x:.2f}<br>Original: " +
                top_5_ro["precio"].astype(str) + " LEI<br>Categoría: " +
                top_5_ro["categoria"] + "<extra></extra>"
))

fig.update_layout(
    title="<b>Top 8 Productos Más Caros - Comparativa en Euros (Con SKU)</b>",
    xaxis_title="Precio (EUR)",
    yaxis_title="Producto",
    barmode="group",
    height=700,
    template="plotly_white",
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

# Añadir anotaciones con las categorías
max_x = max(top_5_es["precio"].max(), top_5_ro["precio_eur"].max())

for _, row in top_5_es.iterrows():
    fig.add_annotation(
        x=row["precio"] + max_x * 0.02,
        y=row["label"],
        text=row["categoria"],
        showarrow=False,
        font=dict(size=10, color="gray"),
        xanchor="left"
    )

for _, row in top_5_ro.iterrows():
    fig.add_annotation(
        x=row["precio_eur"] + max_x * 0.02,
        y=row["label"],
        text=row["categoria"],
        showarrow=False,
        font=dict(size=10, color="gray"),
        xanchor="left"
    )

fig.show()



### Hallazgos interesantes

| Producto                   | España | Rumanía | Diferencia                     |
|---------------------------|--------|---------|--------------------------------|
| Serum Facial Premium      | 19.90€ | 23.84€  | más caro en Rumanía     |
| Crema Facial Reafirmante  | 22.90€ | 21.85€  | más caro en Rumanía      |


La gráfica revela una estrategia de precios inteligente: Rumanía compite en precio en productos masivos, pero se posiciona como premium en productos especializados, mientras España mantiene coherencia en su estructura de precios.

In [31]:
# Top 8 productos más económicos por país 
top_economicos_es = nala_es_ro[nala_es_ro["país"] == "España"].nsmallest(8, "precio").copy()
top_economicos_ro = nala_es_ro[nala_es_ro["país"] == "Rumania"].nsmallest(8, "precio").copy()

# Convertir precios de Rumanía a EUR para comparación
tipo_cambio = 5.03
top_economicos_ro["precio_eur"] = top_economicos_ro["precio"] / tipo_cambio

# Crear etiquetas con SKU para usar en el eje Y
top_economicos_es["label"] = top_economicos_es["nombre"] + " (SKU " + top_economicos_es["SKU"].astype(str) + ")"
top_economicos_ro["label"] = top_economicos_ro["nombre"] + " (SKU " + top_economicos_ro["SKU"].astype(str) + ")"

fig = go.Figure()

# España - barras azules
fig.add_trace(go.Bar(
    y=top_economicos_es["label"],
    x=top_economicos_es["precio"],
    orientation="h",
    name="España",
    marker_color="#1f77b4",
    hovertemplate="<b>%{y}</b><br>Precio: €%{x:.2f}<br>Categoría: " +
                top_economicos_es["categoria"] + "<extra></extra>"
))

# Rumanía - barras naranjas
fig.add_trace(go.Bar(
    y=top_economicos_ro["label"],
    x=top_economicos_ro["precio_eur"],
    orientation="h",
    name="Rumanía (EUR)",
    marker_color="#ff7f0e",
    hovertemplate="<b>%{y}</b><br>Precio: €%{x:.2f}<br>Original: " +
                top_economicos_ro["precio"].astype(str) + " LEI<br>Categoría: " +
                top_economicos_ro["categoria"] + "<extra></extra>"
))

fig.update_layout(
    title="<b>Top 8 Productos Más Económicos - Comparativa en Euros (Con SKU)</b>",
    xaxis_title="Precio (EUR)",
    yaxis_title="Producto",
    barmode="group",
    height=700,
    template="plotly_white",
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

# Añadir anotaciones con las categorías
max_x = max(top_economicos_es["precio"].max(), top_economicos_ro["precio_eur"].max())

for _, row in top_economicos_es.iterrows():
    fig.add_annotation(
        x=row["precio"] + max_x * 0.02,
        y=row["label"],
        text=row["categoria"],
        showarrow=False,
        font=dict(size=10, color="gray"),
        xanchor="left"
    )

for _, row in top_economicos_ro.iterrows():
    fig.add_annotation(
        x=row["precio_eur"] + max_x * 0.02,
        y=row["label"],
        text=row["categoria"],
        showarrow=False,
        font=dict(size=10, color="gray"),
        xanchor="left"
    )

fig.show()

| Producto | Precio España | Precio Rumanía | Diferencia |
|----------|---------------|----------------|------------|
| **Jabón natural** | €2.90 | €1.28 | **-55.9%** en Rumanía |
| **Crema de baño** | €2.35 | €1.77 | **-24.7%** en Rumanía |


En esta grafica podemos ver que los productos mas economicos son, en Rumania el Jabon natural con un costo de 1.28 mientras que en españa cuesta 2.90, y en España lo mas economico es la crema de baño con un costo de 2.35 miestras que en Rumania tiene un costo de 1.77

In [32]:
# Precio promedio por categoría general
# Gregamos tipo de cambio para evitar errores
tipo_cambio = 5.03

# Usar el DataFrame que ya tiene precios en EUR si está disponible
if 'precio_eur' in nala_es_ro.columns:
    df_para_analisis = nala_es_ro
else:
    df_para_analisis = nala_es_ro.copy()
    df_para_analisis['precio_eur'] = df_para_analisis.apply(
        lambda x: x['precio'] if x['país'] == 'España' else x['precio'] / tipo_cambio, 
        axis=1
    )

# Calcular promedios y diferencias
precio_categoria = df_para_analisis.groupby(['categoria_general', 'país'])['precio_eur'].agg(['mean', 'count']).round(2)
precio_categoria = precio_categoria.reset_index()

# Pivot para calcular diferencias
pivot_precios = precio_categoria.pivot(index='categoria_general', columns='país', values='mean')
pivot_precios['diferencia_%'] = ((pivot_precios['Rumania'] - pivot_precios['España']) / pivot_precios['España'] * 100).round(1)

print(pivot_precios)

fig_bar = px.bar(
    precio_categoria,
    x='categoria_general',
    y='mean',
    color='país',
    barmode='group',
    title='Precio Promedio por Categoría General<br><sub>Comparación directa en EUR - Precios convertidos</sub>',
    labels={'mean': 'Precio Promedio (EUR)', 'categoria_general': 'Categoría General'},
    color_discrete_sequence=['#1f77b4', '#ff7f0e'],
    hover_data=['count']
)

# AGREGAR PRECIOS ARRIBA DE LAS BARRAS
fig_bar.update_traces(
    texttemplate='%{y:.2f}€',  # Muestra el precio con 2 decimales y símbolo €
    textposition='outside'     # Coloca el texto fuera de las barras
)

fig_bar.update_layout(
    template='plotly_white',
    xaxis_tickangle=-45,
    showlegend=True,
    yaxis_title='Precio Promedio (EUR)',
    margin=dict(l=50, r=50, t=80, b=100),
    yaxis=dict(
        range=[0, max(precio_categoria['mean']) * 1.15] # Aumenta espacio arriba
    )
)

fig_bar.show()

país               España  Rumania  diferencia_%
categoria_general                               
Cabello              8.97     7.07         -21.2
Corporal             8.66     7.07         -18.4
Ducha y Baño         5.11     3.76         -26.4
Rostro              10.09     8.82         -12.6


Ducha y Baño:

-26.4% más barato en Rumanía

Mayor diferencia (la categoría más favorable para Rumanía)

Cabello:

-21.2% más barato en Rumanía

Corporal:

-18.4% más barato en Rumanía

Rostro:

-12.6% más barato en Rumanía

Menor diferencia (sigue siendo más barato)

In [33]:
# Filtrar las 8 subcategorías más comunes
top_categorias = df_es_ro['categoria'].value_counts().head(8).index

fig_subcat = px.box(
    df_es_ro[df_es_ro['categoria'].isin(top_categorias)],
    x='categoria',
    y='precio',
    color='país',
    title='Precios por Subcategorías Más Comunes',
    points=False
)

fig_subcat.update_layout(
    xaxis_title='Subcategoría',
    yaxis_title='Precio',
    xaxis={'categoryorder':'total descending'},
    height=500
)

fig_subcat.show()

In [35]:
#primero  asegurar la conversión a EUR
tipo_cambio = 5.03

df_con_precio_eur = df_es_ro.copy()
df_con_precio_eur['precio_eur'] = df_con_precio_eur.apply(
    lambda x: x['precio'] if x['país'] == 'España' else x['precio'] / tipo_cambio, 
    axis=1
)

# 2. Calcular promedios en EUR
precio_categoria_eur = df_con_precio_eur.groupby(['categoria_general', 'país'])['precio_eur'].mean().round(2)

# 3. Pivot para calcular diferencias EN EUR
pivot_precios_eur = precio_categoria_eur.reset_index().pivot(
    index='categoria_general', 
    columns='país', 
    values='precio_eur'
)

# 4. Calcular diferencias porcentuales
pivot_precios_eur['diferencia_%'] = (
    (pivot_precios_eur['Rumania'] - pivot_precios_eur['España']) / pivot_precios_eur['España'] * 100
).round(1)

# 5. Ordenar por diferencia absoluta
top_diferencias = pivot_precios_eur.copy()
top_diferencias['diferencia_abs'] = abs(top_diferencias['diferencia_%'])
top_diferencias = top_diferencias.sort_values('diferencia_abs', ascending=False)

# 6. Crear gráfica
fig_top = px.bar(
    top_diferencias.reset_index(),
    y='categoria_general',
    x='diferencia_%',
    title='<b>Top Categorías con Mayor Diferencia de Precio</b><br><sub>Convertido a EUR para comparación real</sub>',
    labels={
        'diferencia_%': 'Diferencia de Precio (%)',
        'categoria_general': 'Categoría'
    },
    color='diferencia_%',
    color_continuous_scale='RdYlBu_r',
    text='diferencia_%',
    orientation='h'
)

# Mejorar formato
fig_top.update_traces(
    texttemplate='%{text:.1f}%',
    textposition='outside'
)

fig_top.update_layout(
    template='plotly_white',
    yaxis={'categoryorder': 'total ascending'},
    xaxis_title='<b>Diferencia de Precio (%)</b>',
    yaxis_title='<b>Categoría</b>',
    coloraxis_showscale=True,
    coloraxis_colorbar=dict(title="Diferencia %"),
    height=500,
    margin=dict(l=50, r=50, t=80, b=50)
)

fig_top.add_annotation(
    text="Valores negativos = Más barato en Rumanía",
    xref="paper", yref="paper",
    x=0.02, y=0.02,
    showarrow=False,
    bgcolor="white",
    bordercolor="black",
    borderwidth=1
)

fig_top.show()

In [47]:
# Crear df_heatmap a partir de tu dataframe combinado
df_heatmap = nala_es_ro_cantidad_valida.copy()

## Aseguramos tipo de cambio
tipo_cambio = 5.03

if 'precio_eur' not in df_heatmap.columns:
    df_heatmap['precio_eur'] = df_heatmap.apply(
        lambda row: row['precio'] if row['país'] == 'España' else row['precio'] / tipo_cambio,
        axis=1
    )

# Preparar datos para heatmap
heatmap_data = df_heatmap.groupby(['categoria_general', 'país'])['precio_eur'].mean().unstack()

fig_heatmap = px.imshow(
    heatmap_data,
    text_auto='.1f',
    aspect="auto",
    color_continuous_scale='YlOrRd',
    title='<b>Mapa de Calor Interactivo - Precios Promedio (EUR)</b><br><sub>Más cálido = Precio más alto</sub>',
    labels=dict(x="País", y="Categoría", color="Precio (EUR)")
)

fig_heatmap.update_layout(
    template='plotly_white',
    height=400
)

fig_heatmap.show()

In [38]:
# Treemap de Distribución de Valor
tipo_cambio = 5.03

# Crear DataFrame temporal con precios normalizados
df_treemap = nala_es_ro.copy()
df_treemap['precio_eur'] = df_treemap.apply(
    lambda x: x['precio'] if x['país'] == 'España' else x['precio'] / tipo_cambio, 
    axis=1
)

fig_treemap = px.treemap(
    df_treemap,
    path=['país', 'categoria_general', 'categoria'],
    values='precio_eur',  # Usar precios en EUR
    color='precio_eur',   # Color basado en EUR
    color_continuous_scale='RdYlGn_r',
    title='Mapa de Árbol - Distribución de Valor por Categorías<br><sub>Tamaño = Precio Total en EUR | Color = Precio Promedio</sub>'
)
fig_treemap.update_layout(
    template='plotly_white',
    height=600
)
fig_treemap.show()

España es el Mercado Principal: La categoría de "España" es mucho más grande que la de "Rumanía", lo que significa que el valor total de ventas (en EUR) es significativamente mayor en España.

Productos Corporales y Faciales son los Más Valiosos: Tanto en España como en Rumanía, las categorías de "Corporal" y "Rostro" son las que generan más ingresos, siendo las secciones más grandes dentro de cada país.


Productos Destacados: En ambos países, productos específicos como "Crema Facial" y "Exfoliante Corporal" se ven como componentes importantes dentro de sus categorías, lo que indica que son artículos clave en las ventas.

En resumen, la estrategia de ventas se centra con mayor fuerza en España y en los productos de cuidado para la piel del cuerpo y el rostro.



In [39]:
# 1. Crear columna "gama" según país
def clasificar_gama(row):
    precio = row["precio"]
    pais = row["país"]

    # España → euros
    if pais.lower() == "españa":
        if precio <= 8:
            return "Baja"
        elif precio <= 15:
            return "Media"
        else:
            return "Alta"

    # Rumanía → lei
    elif pais.lower() == "rumania":
        if precio <= 40:
            return "Baja"
        elif precio <= 70:
            return "Media"
        else:
            return "Alta"

    return None

df_es_ro["gama"] = df_es_ro.apply(clasificar_gama, axis=1)

# 2. Agrupar datos por categoría y gama

df_gama = (
    df_es_ro.groupby(["categoria_general", "gama"])
    .size()
    .reset_index(name="cantidad")
)

# 3. Gráfico dinámico Plotly
fig = px.bar(
    df_gama,
    x="categoria_general",
    y="cantidad",
    color="gama",
    barmode="group",
    title="Segmentación por gama de precio (NALA España & Rumanía)",
    labels={
        "cantidad": "Cantidad de productos",
        "categoria_general": "Categoría",
        "gama": "Gama de precio"
    },
)

# Mostrar valores arriba de las barras
fig.update_traces(texttemplate="%{y}", textposition="outside")

fig.update_layout(
    template="plotly_white",
    xaxis_title="Categoría",
    yaxis_title="Productos",
    legend_title="Gama",
    uniformtext_minsize=8,
    uniformtext_mode="hide",
    
)

fig.show()

La gama MEDIA domina el mercado: Es la más vendida en todas las categorías, especialmente en Corporal y Rostro.

Rostro es la categoría más importante: Tiene la mayor cantidad de productos totales (219), liderando tanto en gama Media como Alta.

Cabello tiene poca variedad de precios: Solo ofrece productos de gama Baja y Media, sin opciones premium.

Corporal es la segunda categoría clave: Con 115 productos en gama Media, muestra una fuerte presencia en el segmento de precio medio.

En resumen: La estrategia se centra en productos de precio medio, siendo el cuidado facial la categoría principal y más diversa en gamas de precio.

In [50]:
# Violin plot por categoría general con precios convertidos
# Asegurar que existe el tipo de cambio
tipo_cambio = 5.03  # 1 EUR = 5.03 RON

# Copiar el DF original que tienes cargado
df_es_ro_conv = nala_es_ro_cantidad_valida.copy()

# Convertir precios a euros según el país
df_es_ro_conv['precio_eur'] = df_es_ro_conv.apply(
    lambda row: row['precio'] / tipo_cambio if row['país'] == 'Rumanía' else row['precio'],
    axis=1
)

fig_violin_cat = px.violin(
    df_es_ro_conv, #Conversion de moneda
    x='categoria_general',
    y='precio_eur',
    color='país',
    box=True,
    points=False,
    title='Distribución de Precios por Categoría General (EUR)',
    labels={'precio_eur': 'Precio (EUR)', 'categoria_general': 'Categoría General'}
)
fig_violin_cat.show()

## Resumen Ejecutivo

In [41]:
# Gráfica Comparativa - Métricas Clave
fig_comparativa = go.Figure()

# Valores calculados
valores_es = [
    df_es['precio'].median(),
    productos_salario_es,
    1.0,
    (df_es['precio'].std()/df_es['precio'].mean())*100,
    0
]

valores_ro = [
    df_ro['precio'].median()/tipo_cambio,
    productos_salario_ro,
    productos_salario_es/productos_salario_ro,
    (df_ro['precio'].std()/df_ro['precio'].mean())*100,
    ((df_ro['precio'].median()/tipo_cambio) / df_es['precio'].median() - 1)*100
]

metricas = ['Precio Mediano (EUR)', 'Productos con Salario', 'Accesibilidad', 'Dispersión', 'Diferencia %']

# Barras España
fig_comparativa.add_trace(go.Bar(
    name='España',
    x=metricas,
    y=valores_es,
    marker_color='blue',
    text=[f'€{valores_es[0]:.2f}', f'{valores_es[1]:.0f}', f'{valores_es[2]:.1f}x', 
        f'{valores_es[3]:.1f}%', f'{valores_es[4]:.1f}%'],
    textposition='auto'
))

# Barras Rumanía
fig_comparativa.add_trace(go.Bar(
    name='Rumanía',
    x=metricas,
    y=valores_ro,
    marker_color='red',
    text=[f'€{valores_ro[0]:.2f}', f'{valores_ro[1]:.0f}', f'{valores_ro[2]:.1f}x', 
        f'{valores_ro[3]:.1f}%', f'{valores_ro[4]:.1f}%'],
    textposition='auto'
))

fig_comparativa.update_layout(
    title='Métricas Clave - España vs Rumanía',
    xaxis_title='Métricas',
    yaxis_title='Valor',
    barmode='group'
)

fig_comparativa.show()

# 🧾 Conclusiones del análisis comparativo Nala España vs Rumanía

---

## ✅ Resumen ejecutivo — Respuesta a las hipótesis

- **Hipótesis principal (accesibilidad):** ❌ **No se cumple.**  
  Aunque los precios nominales en España son ligeramente más altos , al considerar el poder adquisitivo (salario mínimo), la **accesibilidad es prácticamente igual** (~0.7% del salario mínimo en ambos países).  
  En términos de poder de compra, un salario mínimo permite comprar aproximadamente **150 productos en España** y **135 en Rumanía**, por lo que España es solo **~11% más accesible**.

- **Hipótesis secundaria (estructura de precios / mix):** ⚠️ **Parcialmente se cumple.**  
  El mix de producto y la gama media dominan en ambos mercados, pero **Rumanía presenta mayor dispersión de precios** y algunos productos premium son **más caros en Rumanía**, como el *Serum Facial Premium*.

---

## 1️⃣ Análisis general de mercado

- **Distribución de precios:**  
  - España: precios más concentrados y estables.  
  - Rumanía: precios más heterogéneos, con presencia de productos muy baratos y algunos más caros.  

- **Rango y dispersión:**  
  - Rumanía: rango más amplio y dispersión.  
  - España: dispersión mas estable.  
  → Diferencia significativa en variabilidad, pero no en accesibilidad.

- **Conclusión:**  
  Rumanía tiene una estructura de precios menos homogénea, con más contrastes entre productos económicos y premium.

---

## 2️⃣ Análisis por categorías

- **Categorías con precios más altos:**  
  - *Rostro*: mayor presencia de productos premium.  
  - *Corporal*: segunda categoría en valor promedio.  

- **Categorías con precios más bajos:**  
  - *Cabello*: precios estables y más bajos, sin gama alta.  

- **Variación entre países:**  
  - *Rostro* muestra las diferencias más marcadas (productos premium más caros en RO).  
  - *Cabello* mantiene precios similares en ambos mercados.  

---

## 3️⃣ Análisis de productos

- **Productos más caros (comparaciones destacadas):**  
  - Serum Facial Premium      - E 19.90€ - R 23.84€  - más caro en Rumanía
  - Crema Facial Reafirmante  - E 22.90€ - R 21.85€  - más caro en Rumanía

- **Productos más económicos:**  
  - *Jabón natural* - E €2.90 - R €1.28 - mas economico en Rumania
  - *Crema de baño* - E €2.35 - R €1.77 - mas economico en Rumania

- **Relación precio-cantidad:**  
  - No siempre proporcional; algunos envases pequeños tienen precios unitarios más altos (posicionamiento premium).  

- **Productos idénticos con precios distintos:**  
  - Sí existen diferencias para el mismo SKU o nombre (ejemplo: *Serum Facial Premium*).  

---

## 4️⃣ Análisis de accesibilidad (poder adquisitivo)

**Considerando los salarios mínimos, ¿es Nala igual de accesible en ambos mercados?**  
- **NO**, pero la diferencia es menor de lo esperado  
- **España**: 0.67% del salario mínimo por producto  
- **Rumanía**: 0.74% del salario mínimo por producto  
- **Ratio de accesibilidad**: 1.11x (ES/RO) → España 11% más accesible  
- **Productos comprables con salario mínimo**:  
  - España: ~150 productos  
  - Rumanía: ~135 productos

- **Conclusión:**  
  Las diferencias de accesibilidad son **mínimas**. España es solo **~11% más accesible**, sin evidencia de que Rumanía sea significativamente menos accesible.



## 🧩 Conclusión general
**Nala mantiene precios similares en ambos países cuando consideramos el poder adquisitivo de la gente.**

**Lo que encontramos:**
- 💰 **En Rumanía** los precios son más bajos, pero el salario también es menor
- 💰 **En España** los precios son más altos, pero el salario es mayor
- ⚖️ **Resultado**: La marca es casi igual de accesible en ambos países

**Estrategia de Nala:**
- 🏷️ **Productos básicos**: Más baratos en Rumanía
- 💎 **Productos premium**: Más caros en Rumanía
- 📊 **En general**: Precios coherentes entre mercados

**Conclusión principal:**
Nala ha logrado equilibrar sus precios para que sean similares en accesibilidad, adaptándose a la realidad económica de cada país sin perder su esencia de marca.